# Diseño y Vlsm 

# 📘 Documentación Técnica Final - VLSM Continuo

## 1. Criterio de Diseño
* **Fórmula Base:** $200 + (10 \times N_{red})$.
* **Carga Real:** Se suman los dispositivos físicos (IoT, Servidores) de cada sala.
* **Continuidad:** El factor $N_{red}$ **NO** se reinicia entre sedes (1 al 25), lo que provoca que a partir de la red #6, todas las subredes requieran máscaras **/23** para soportar la base teórica de >254 hosts.

---

## 2. Direccionamiento por Sede

### 📍 2.1 RÍO DE JANEIRO (Redes 1 - 7)
**Bloque:** `10.2.64.0`
**Factores:** 1 al 7.

| Orden | VLAN Nombre | ID | Cálculo (Factor + Carga Real) | Total | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 1 | **Admin** | 10 | $200 + 10(1) + \mathbf{7 \text{ IoT}}$ | **217** | **/24** |
| 2 | **Logistica** | 20 | $200 + 10(2) + \mathbf{7 \text{ IoT}}$ | **227** | **/24** |
| 3 | **Prensa** | 30 | $200 + 10(3) + \mathbf{5 \text{ IoT}}$ | **235** | **/24** |
| 4 | **Seguridad** | 40 | $200 + 10(4) + \mathbf{7 \text{ IoT}}$ | **247** | **/24** |
| 5 | **Inalambrica**| 50 | $200 + 10(5) + \text{Móviles}$ | **>254** | **/23** |
| 6 | **Servidores**| 60 | $200 + 10(6) + \mathbf{13 \text{ Eq.}}$ | **273** | **/23** |
| 7 | **Manager** | 99 | $200 + 10(7) \text{ (Gestión)}$ | **270** | **/23** |

<br>

**📝 Tabla de Direccionamiento IP (Río de Janeiro):**

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/24** | `10.2.64.0` | `.64.2` a `.64.254` | `10.2.64.1` |
| **20** | Logistica | **/24** | `10.2.65.0` | `.65.2` a `.65.254` | `10.2.65.1` |
| **30** | Prensa | **/24** | `10.2.66.0` | `.66.2` a `.66.254` | `10.2.66.1` |
| **40** | Seguridad | **/24** | `10.2.67.0` | `.67.2` a `.67.254` | `10.2.67.1` |
| **50** | Inalambrica| **/23** | `10.2.68.0` | `.68.2` a `.69.254` | `10.2.68.1` |
| **60** | Servidores| **/23** | `10.2.70.0` | `.70.2` a `.71.254` | `10.2.70.1` |
| **99** | Manager | **/23** | `10.2.72.0` | `.72.2` a `.73.254` | `10.2.72.1` |

---

### 📍 2.2 SÃO PAULO (Redes 8 - 13)
**Bloque:** `10.2.74.0`
**Factores:** 8 al 13.

| Orden | VLAN Nombre | ID | Cálculo (Factor + Carga Real) | Total | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 8 | **Admin** | 10 | $200 + 10(8) + 0 \text{ IoT}$ | **280** | **/23** |
| 9 | **Logistica** | 20 | $200 + 10(9) + 0 \text{ IoT}$ | **290** | **/23** |
| 10 | **Prensa** | 30 | $200 + 10(10) + 0 \text{ IoT}$ | **300** | **/23** |
| 11 | **Seguridad** | 40 | $200 + 10(11) + 0 \text{ IoT}$ | **310** | **/23** |
| 12 | **Inalambrica**| 50 | $200 + 10(12) + \mathbf{4 \text{ Hardw.}}$ | **324** | **/23** |
| 13 | **Manager** | 99 | $200 + 10(13) \text{ (Gestión)}$ | **330** | **/23** |

<br>

**📝 Tabla de Direccionamiento IP (São Paulo):**

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/23** | `10.2.74.0` | `.74.2` a `.75.254` | `10.2.74.1` |
| **20** | Logistica | **/23** | `10.2.76.0` | `.76.2` a `.77.254` | `10.2.76.1` |
| **30** | Prensa | **/23** | `10.2.78.0` | `.78.2` a `.79.254` | `10.2.78.1` |
| **40** | Seguridad | **/23** | `10.2.80.0` | `.80.2` a `.81.254` | `10.2.80.1` |
| **50** | Inalambrica| **/23** | `10.2.82.0` | `.82.2` a `.83.254` | `10.2.82.1` |
| **99** | Manager | **/23** | `10.2.84.0` | `.84.2` a `.85.254` | `10.2.84.1` |

---

### 📍 2.3 SALVADOR (Redes 14 - 19)
**Bloque:** `10.2.86.0`
**Factores:** 14 al 19.

| Orden | VLAN Nombre | ID | Cálculo (Factor + 26 IoTs Visuales) | Total | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 14 | **Admin** | 10 | $200 + 10(14) + \mathbf{7 \text{ IoT}}$ | **347** | **/23** |
| 15 | **Logistica** | 20 | $200 + 10(15) + \mathbf{7 \text{ IoT}}$ | **357** | **/23** |
| 16 | **Prensa** | 30 | $200 + 10(16) + \mathbf{5 \text{ IoT}}$ | **365** | **/23** |
| 17 | **Seguridad** | 40 | $200 + 10(17) + \mathbf{7 \text{ IoT}}$ | **377** | **/23** |
| 18 | **Inalambrica**| 50 | $200 + 10(18) + \mathbf{1 \text{ Srv}}$ | **381** | **/23** |
| 19 | **Manager** | 99 | $200 + 10(19) \text{ (Gestión)}$ | **390** | **/23** |

<br>

**📝 Tabla de Direccionamiento IP (Salvador):**

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/23** | `10.2.86.0` | `.86.2` a `.87.254` | `10.2.86.1` |
| **20** | Logistica | **/23** | `10.2.88.0` | `.88.2` a `.89.254` | `10.2.88.1` |
| **30** | Prensa | **/23** | `10.2.90.0` | `.90.2` a `.91.254` | `10.2.90.1` |
| **40** | Seguridad | **/23** | `10.2.92.0` | `.92.2` a `.93.254` | `10.2.92.1` |
| **50** | Inalambrica| **/23** | `10.2.94.0` | `.94.2` a `.95.254` | `10.2.94.1` |
| **99** | Manager | **/23** | `10.2.96.0` | `.96.2` a `.97.254` | `10.2.96.1` |

---

### 📍 2.4 RECIFE (Redes 20 - 25)
**Bloque:** `10.2.98.0`
**Factores:** 20 al 25.

| Orden | VLAN Nombre | ID | Cálculo (Factor + Carga Real) | Total | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 20 | **Admin** | 10 | $200 + 10(20) + 0 \text{ IoT}$ | **400** | **/23** |
| 21 | **Logistica** | 20 | $200 + 10(21) + 0 \text{ IoT}$ | **410** | **/23** |
| 22 | **Prensa** | 30 | $200 + 10(22) + 0 \text{ IoT}$ | **420** | **/23** |
| 23 | **Seguridad** | 40 | $200 + 10(23) + 0 \text{ IoT}$ | **430** | **/23** |
| 24 | **Inalambrica**| 50 | $200 + 10(24) + \mathbf{1 \text{ Srv}}$ | **441** | **/23** |
| 25 | **Manager** | 99 | $200 + 10(25) \text{ (Gestión)}$ | **450** | **/23** |

<br>

**📝 Tabla de Direccionamiento IP (Recife):**

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/23** | `10.2.98.0` | `.98.2` a `.99.254` | `10.2.98.1` |
| **20** | Logistica | **/23** | `10.2.100.0` | `.100.2` a `.101.254` | `10.2.100.1` |
| **30** | Prensa | **/23** | `10.2.102.0` | `.102.2` a `.103.254` | `10.2.102.1` |
| **40** | Seguridad | **/23** | `10.2.104.0` | `.104.2` a `.105.254` | `10.2.104.1` |
| **50** | Inalambrica| **/23** | `10.2.106.0` | `.106.2` a `.107.254` | `10.2.106.1` |
| **99** | Manager | **/23** | `10.2.108.0` | `.108.2` a `.109.254` | `10.2.108.1` |

---

## 3. Enlaces WAN (Backbone)
**Bloque Asignado:** `10.2.110.0`
*(Debido a que Recife termina en la IP .109.255, el backbone empieza en la red .110)*

**Máscara:** `/30` (255.255.255.252)

| Enlace | Router (DCE - Reloj) | IP | Router (DTE) | IP | Dirección de Red |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **Río -> SP** | Río `Se0/3/0` | **`.110.1`** | SP `Se0/3/0` | **`.110.2`** | `10.2.110.0/30` |
| **Río -> Sal** | Río `Se0/2/0` | **`.110.5`** | Sal `Se0/2/0` | **`.110.6`** | `10.2.110.4/30` |
| **Río -> Rec** | Río `Se0/3/1` | **`.110.9`** | Rec `Se0/2/0` | **`.110.10`**| `10.2.110.8/30` |
| **SP -> Rec** | SP `Se0/3/1` | **`.110.13`**| Rec `Se0/3/0` | **`.110.14`**| `10.2.110.12/30` |
| **Sal -> Rec** | Sal `Se0/3/1` | **`.110.17`**| Rec `Se0/2/1` | **`.110.18`**| `10.2.110.16/30` |